# Data Cleaning - AI Salary Analysis: Remote Work and Experience Premiums

## Purpose

This notebook prepares the raw AI salary data for analysis by:
1. Filtering to the US market and full-time positions 
2. Creating clean remote work classifications 
3. Categorizing job roles for hypothesis testing 
4. Removing outliers and selecting releavnt variables 

**input:** 'salaries.csv'

**Output:** 'cleaned_salaries.csv'

## Load Raw Data and Import Libraries